In [10]:
import pandas as pd
import openai

# Set your OpenAI API key 
openai.api_key = "YOUR_OPENAI_API_KEY"


# Function to read data from the Excel file and store it as a dictionary
def load_user_profile(file_name):
    try:
        df = pd.read_excel(file_name)
        user_profile = {row['Question']: row['Answer'] for _, row in df.iterrows()}
        return user_profile
    except Exception as e:
        print(f"Error loading user profile from file: {e}")
        return None

    
# Function to get career recommendations from GPT-4 model
def suggest_career_paths(user_profile):
    
    if not user_profile:
        return "User profile data is missing or invalid."
    
    # Construct a structured prompt using the user profile data
    prompt = """
    Based on the following user profile, suggest 3 career paths that are realistic and aligned with the user's background. 
    Ensure these are logical career transitions instead of random suggestions. Each recommendation should be supported with brief reasoning. 
    
    # 1. Personality, Strengths, and Skills
    - MBTI Personality Type: {personality_type}
    - Top Strengths: {strengths_1}, {strengths_2}, {strengths_3}
    - Best Tasks in Previous Work: {previous_work_skills}
    
    # 2. Interests and Passions
    - Activities Where Time Flies: {passions}
    - Unlimited Task: {unlimited_time_task}
    - Energizing Activities: {energy_sources}
    
    # 3. Values, Impact, and Purpose
    - Top 3 Life Values: {values}
    - Desired Problem to Solve: {impact_purpose}
    
    # 4. Current Situation and Goal Analysis
    - Education Background: {education}
    - Career Background: {career_background}
    - Current Situation: {current_situation}
    - Desired Future Position: {where_to_be}
    
    # 5. Work Preferences
    - Work Preferences: {work_preferences}
    - Preferred Work Type: {work_type}
    - Problem You Solve at Work: {work_output}
    - Career Priorities:
        1. {priority_1}
        2. {priority_2}
        3. {priority_3}

    Consider the following constraints:
    - Ensure recommendations align with the preferred work type (e.g., avoid 9-5 roles for someone seeking entrepreneurship, avoid freelance and entreprenurial roles if looking for a 9-5).
    - Focus on career paths that build upon their previous work experience for a smoother transition. Avoid suggesting roles that requires completely different work experience.
    - Don't recommend roles that dont exist much, stick to known and popular role titles.
    Provide recommendations that are achievable career transitions rather than generic roles. Explain how each career fits the person’s background and aspirations. Keep the response under 250 words.    
    """.format(
        personality_type=user_profile['What is your MBTI personality type ?'],
        strengths_1=user_profile['What is one of your top strengths or skills (1/3)?'],
        strengths_2=user_profile['What is one of your top strengths or skills (2/3)?'],
        strengths_3=user_profile['What is one of your top strengths or skills (3/3)?'],
        previous_work_skills=user_profile['What were some of the best tasks you\'ve done in your previous work?'],
        passions=user_profile['What makes you feel like time flies by?'],
        unlimited_time_task=user_profile['What would you do if there were no limits?'],
        energy_sources=user_profile['What energizes you the most?'],
        interest_domains=user_profile['Which domains you are interested the most?'],
        best_medium=user_profile['Which medium you are best at?'],
        values=user_profile['What are your top 3 values in life?'],
        impact_purpose=user_profile['What problem would you want to solve in the world and your life?'],
        contribution=user_profile['How can you contribute to your life purpose through your career?'],
        education=user_profile['What is your education background?'],
        career_background=user_profile['What is your career background?'],
        current_situation=user_profile['What is your current situation?'],
        where_to_be=user_profile['Where do you want to be?'],
        work_preferences=user_profile['what is your work preferences? (culture, salary, benefits, logistics)'],
        work_type= user_profile['What is your preferred work type? (9-5, entrepreneurship, freelance)'],
        work_output=user_profile['What problem you solve at work? (the output of your work)'],
        priority_1=user_profile['What is your top prioritity in your next career move? (1/3)?'],
        priority_2=user_profile['What is your top prioritity in your next career move? (2/3)?'],
        priority_3=user_profile['What is your top prioritity in your next career move? (3/3)?']
    )
    
    # Send the user profile data in a structured chat format
    messages = [
        {"role": "system", "content": "You are a professional career advisor AI, specializing in scientifically backed career transitions."},
        {"role": "user", "content": prompt},  # Kullanıcının profilini içeren mesaj
    ]
    
    # Fetch career suggestions using the AI model
    try:
        response = openai.ChatCompletion.create(
            model="gpt-4o-mini",
            messages=messages,
            temperature=0.6,
            max_tokens=300,
            top_p=1,
            frequency_penalty=0,
            presence_penalty=0
        )
        return response.choices[0].message['content']
    except Exception as e:
        print(f"Error generating career suggestions: {e}")
        return None
    
    # Return the career suggestions
    return response.choices[0].message.content
    

# Function that ties everything together and takes the file name as an argument
def main(file_name):
    # Load user profile from the Excel file
    user_profile = load_user_profile(file_name)
    
    # Get career suggestions based on the user's profile
    career_suggestions = suggest_career_paths(user_profile)
    
    if career_suggestions:
        # Print the career suggestions
        print("\nHere are your top 3 career path recommendations:\n")
        print(career_suggestions)
    else:
        print("Could not generate career suggestions.")

In [ ]:
# Run the tool
if __name__ == "__main__":
    
    # Define the file path of xlsx file (template can be found in the github repo)
    file_path = 'YOUR_FILE_PATH' # ensure the correct file path is provided
    
    # Call the main function with the file path
    main(file_path)  # Pass the file path to the main function